In [1]:
import pandas as pd
import os
import plotly.graph_objects as go
from datetime import datetime as dt
import numpy as np
from dash import dcc,html
from jupyter_dash import JupyterDash

In [2]:
data_dir = 'C:/Users/gq19765/SyncThing/Repositories/LABMARCS/PatientViz_Sep22/processed_data/'
output_dir = 'C:/Users/gq19765/SyncThing/Repositories/LABMARCS/PatientViz_Sep22/output/'
os.chdir(data_dir)

In [3]:
dem = pd.read_csv('dem.csv')
outcomes = pd.read_csv('outcomes.csv')

masterlist = pd.read_csv('masterlist.update.csv')

labs = pd.read_csv('labs.csv')
labs['Measure'] = labs['Measure'].astype('category')
category_order_list = ['BNP','LDH','HBA1c','trig','trop','DDM','FER','PCT','fib','Glucose','PT','APTT','PLT','CRP','eGFR','poctLAC','poctpH','O2','CO2','WCC','Lymphocytes','Neutrophils','NLR','HB','BE']
labs['Measure'] = labs['Measure'].cat.set_categories(category_order_list, ordered=True)
labs = labs.sort_values(by='Measure')
avoncap = pd.read_csv('avoncap.csv')
bc = pd.read_csv('blood_culture.csv')
virology = pd.read_csv('virology.csv')
covidCT = pd.read_csv('covidCT.csv')

In [4]:
# Dummy Measures data to ensure all categories are plotted, and in correct order
dummy_values = [np.nan]*25
dummy_dates = [0]*25
dummy_measures = ['BE', 'HB', 'NLR', 'Neutrophils', 'Lymphocytes', 'WCC', 'CO2', 'O2', 'poctpH', 'poctLAC', 'eGFR', 'CRP', 'PLT', 'APTT', 'PT', 'Glucose', 'fib', 'PCT', 'FER', 'DDM', 'trop', 'trig', 'HBA1c', 'LDH', 'BNP']
dummy_data = pd.DataFrame({'Numeric.Result':dummy_values,'date':dummy_dates,'Measure':dummy_measures})

## `patient_viz` function
### Parameters
- `id`: The patient ID
- `label_type`:
    - 'graph' where key event labels are on the plot itself
    - 'legend' where key event labels are in the legend, sorted chronologically

In [5]:
def patient_viz(id,label_type):
    # Ensure dates list has at least one value, and calculate difference in days between date and admission
    def vali_date(query):
        if len(query) > 0: 
            days = []
            for date in query:
                days.append((dt.strptime(date,"%Y-%m-%d") - dt.strptime(first_adm,"%Y-%m-%d"))/pd.Timedelta(days=1))
            return days
        else:
            return pd.Series(dtype='int')
    # Get admission date as reference for days since admission
    # Outcomes for patient
    p_outcome = outcomes[outcomes.uob_ID==id]
    # Grab and sort admission dates
    first_adm = p_outcome['admissionDate'].dropna().tolist()[0]

    p_labs = labs[labs.uob_ID==id]
    p_labs['date'] = vali_date(p_labs['date'])
    p_labs = pd.concat([dummy_data,p_labs],axis=0)

    # Key event dates
    # Each key event specifies a colour, name and date, and will be plotted as a dashed line. 
    
    adm_d = {'Dates':vali_date(p_outcome['admissionDate'].dropna().tolist()),'Colour':'#1e90ff','name':'Admission to Hospital','dash':'dot','offset':0.1}
    covid_d = {'Dates':vali_date(covidCT[covidCT.uob_ID==id].date.dropna().tolist()),'Colour':'#ff8c00','name':'Covid +ve date','dash':'dot','offset':0}
    icu_adm_d = {'Dates':vali_date(p_outcome['ITU_Start'].dropna().tolist()),'Colour':'#0000ff','name':'Admission to ICU','dash':'dot','offset':0.1}
    icu_dis_d = {'Dates':vali_date(p_outcome['ITU_End'].dropna().tolist()),'Colour':'#66cdaa','name':'Discharge from ICU','dash':'dot','offset':-0.1}
    death_d = {'Dates':vali_date(p_outcome['deathDate'].dropna().tolist()),'Colour':'#ff1493','name':'Date of Death','dash':'dot','offset':0}
    dis_d = {'Dates':vali_date(p_outcome['dischargeDate'].dropna().tolist()),'Colour':'#00ff00','name':'Date of Discharge','dash':'dot','offset':0.1}
    # Collect all key events into 'outcomes_list'
    outcomes_list = [covid_d,adm_d,icu_adm_d,icu_dis_d,death_d,dis_d]



    # Sort by date function, to plot traces/legend in chronological order
    def sortByDate(e):
        return min(e['Dates'], default=100)

    if label_type == "legend":
        outcomes_list.sort(key=sortByDate)
        

    # Plot lab results
    fig = go.Figure(data=go.Heatmap(
        z=p_labs['Numeric.Result'], # Value of biomarker
        x=p_labs['date'], # Date of biomarker reading
        y=p_labs['Measure'], # Biomarker
        colorscale='thermal', # Shows most contrast IMO
        ygap=1, # Spacing between bricks for clarity
        colorbar=dict(yanchor="top", y=1, x=-0.28), # Offset to stop overlap of colorbar with y-axis title
        connectgaps=False 
    ))

    # Plot key dates
    # For each date, plot a dashed line, with
    if label_type == "legend":
        for outcome in outcomes_list:
            for date in outcome['Dates']:
                fig.add_trace(
                    go.Scatter(
                        x=[date-outcome['offset'],date-outcome['offset']],
                        y=['BNP','BE'], # Dashed line between bottomost and uppermost values on y-axis
                        line_width=2,
                        line_dash=outcome['dash'],
                        line_color=outcome['Colour'],
                        name = outcome['name'],
                        mode='lines'
                    )
                )         

    if label_type == "graph":
        for outcome in outcomes_list:
            if(len(outcome['Dates']) > 0):
                for date in outcome['Dates']:
                    fig.add_vline(
                        x=date,
                        line_width=2,
                        line_dash="dash",
                        line_color=outcome['Colour'],
                        annotation={
                            "text":outcome['name'],
                            "textangle":45,
                            "font_color":outcome['Colour'],
                            "opacity":0.5,
                            "bgcolor":"white",
                            "bordercolor":outcome['Colour'],
                            "borderwidth":2,
                            "opacity":0.7,
                            "xanchor":"right", # x and yanchor make sure the bottom right of the text box touch the relevant line
                            "yanchor":"bottom",
                            "xshift":14 # because of angle of textbox, xshift needed to align corner with line 
                            })
            
    range_lower = min(p_labs['date']) - 1
    range_upper = max(p_labs['date']) + 2
            
    fig.update_layout(
        title='Patient Visualisation',
        autosize = False,
        width = 1100,
        height = 600,
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4
        ),
        yaxis={
            "title":'Biomarker',
            "title_standoff":0,
            "tickson":"boundaries", # Sets color brick to have gridlines either side, instead of in centre of brick.
            "dtick":1 # Ensures all category labels are shown
            },
        xaxis={
            "title":'Day (relative to admission date)',
            "minor_griddash":"dot",
            "range":[range_lower,range_upper]
            }
    )
    app = JupyterDash(__name__)
    server = app.server
    app.layout = html.Div([
        dcc.Graph(figure = fig)
    ])
    #app.run_server(mode='inline',debug=True)
    file_name = "patient-viz_%s.svg" % id
    fig.write_image(file_name)

In [ ]:
# patient_examples = ['8810','05a4','23f0','3075','1b49','1a99','0adc']
os.chdir(output_dir)
for patient in masterlist['uob_ID']:
    try:
        patient_viz(patient,'legend')
    except:
        continue

In [6]:
patient_viz('002f','legend')

C:\Users\gq19765\AppData\Local\Temp\ipykernel_14556\1192023823.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_labs['date'] = vali_date(p_labs['date'])


In [ ]:
test = "patient-viz_%s.svg" % '002f'
print(test)